In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import random
import re
import pdb
import pickle

## Ulta Scraping
### Exploration of the soup 

In [3]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://www.ulta.com/hair-styling-products?N=26xf&Nrpp=96&Ns=product.bestseller%7C1', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta content="0iJgVZRwJEqcwh-DfHO3TnKtFasUWdJmJbVBo2j8eg0" name="google-site-verification"/>
<title>Styling Products | Ulta Beauty</title>
<meta content="Styling Products | Ulta Beauty" name="title"/>
<meta content="Oracle Commerce reference application." name="description"/>
<link href="https://www.ulta.com/hair-styling-products?N=26xf" rel="canonical"/>
<link href="/ui/static/css/vendor.1566326797713.css" rel="stylesheet"/><link href="/ui/static/css/mhp.1566326797713.css" rel="stylesheet"/><link href="//statica.ultainc.com/static/css/combineCommonHeader.css" rel="stylesheet" type="text/css">
<link href="//statica.ultainc.com/static/css/combineHeader.css" rel="stylesheet" type="text/css">
<scri

In [ ]:
xdesc = soup.find_all("p", class_='prod-desc')[95].text
xtitle = soup.find_all("h4", class_="prod-title")[0].text

In [ ]:
xdesc

In [ ]:
xlist = []
xlist.append(xtitle + xdesc)
xlist

In [ ]:
s = re.sub(r"\n", "", xlist[0])
s = re.sub(r"\t", "", s)
xlist.append(s)
xlist

In [ ]:
# find how to increment pages
#page 1
https://www.ulta.com/hair-styling-products?N=26xf&Nrpp=96&Ns=product.bestseller%7C1
#page 2
https://www.ulta.com/hair-styling-products?N=26xf&No=96&Nrpp=96&Ns=product.bestseller%7C1
#page 3
https://www.ulta.com/hair-styling-products?N=26xf&No=192&Nrpp=96&Ns=product.bestseller%7C1
https://www.ulta.com/hair-styling-products?N=26xf&No=1056&Nrpp=96&Ns=product.bestseller%7C1    

In [ ]:
list(range(0,1056+1,96))

In [ ]:
https://www.ulta.com/hair-shampoo-conditioner?N=27ih
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=96&Nrpp=96
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=192&Nrpp=96
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=1440&Nrpp=96    

In [ ]:
list(range(0,1440+1,96))

In [ ]:
with open('ulta_products.pkl', 'rb') as f:
    pickle.dump

### Scrape funcitonality

In [4]:
# Scrape the Ulta Styling products section

ulta_styling = []
for i in list(range(0,1056+1,96)):
    url = f'https://www.ulta.com/hair-styling-products?N=26xf&No={i}&Nrpp=96&Ns=product.bestseller%7C1'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
    print(html_page.status_code)
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    
    
    for x in range(0,96):
        try:
            xdesc = soup.find_all("p", class_='prod-desc')[x].text
            xtitle = soup.find_all("h4", class_="prod-title")[x].text
            
            # do some cleaning
            s = re.sub(r"\n", "", xdesc)
            s = re.sub(r"\t", "", s)
            new_xdesc = s

            s2 = re.sub(r"\n", "", xtitle)
            s2 = re.sub(r"\t", "", s2)
            new_xtitle = s2


            ulta_styling.append(new_xtitle + " " + new_xdesc)
        
        except:    
            continue
        

200
200
200
200
200
200
200
200
200
200
200
200


In [5]:
ulta_styling[0:20]

['Tigi Bed Head Masterpiece Shine Hairspray',
 'Drybar The Blowout Brigade Set',
 'Kenra Professional Volume Spray 25',
 'Tigi Bed Head After-Party',
 'Sexy Hair Big Sexy Hair Spray & Play Volumizing Hairspray',
 'Sebastian Shaper Plus Extra Hold Hairspray',
 'Kenra Professional Platinum Blow-Dry Spray',
 "Bumble and bumble Bb.Hairdresser's Invisible Oil Heat/UV Protective Primer",
 'Sexy Hair Big Sexy Hair Spray & Play Harder Firm Volumizing Hairspray',
 'Living Proof Online Only No Frizz Favorites Jumbo Bundle',
 'Tigi Bed Head Hard Head Hairspray',
 'Sexy Hair Big Sexy Hair Root Pump Plus Humidity Resistant Volumizing Spray Mousse',
 'DevaCurl Ultra Defining Gel Strong Hold No-Crunch Styler',
 'Curlsmith Curl Conditioning Oil-In-Cream',
 'Tigi Catwalk Curls Rock Amplifier',
 'Curlsmith Curl Quenching Conditioning Wash',
 'Redken Control Addict 28 Extra High-Hold Hairspray',
 'AG Hair Curl Re:Coil Curl Activator',
 'Ouidad Advanced Climate Control Heat & Humidity Gel',
 'Sebastian Sh

In [6]:
len(ulta_styling)

1145

In [ ]:
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=0&Nrpp=9
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=9&Nrpp=9
    

In [8]:
# Scrape the Ulta Shampoo and conditioner products section

ulta_cleansers = []
for i in list(range(0,1458+1,9)):
    url = f'https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No={i}&Nrpp=9'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
    print(html_page.status_code)
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    
    
    for x in range(0,9):
        try:
            xdesc = soup.find_all("p", class_='prod-desc')[x].text
            xtitle = soup.find_all("h4", class_="prod-title")[x].text
            
            # do some cleaning
            s = re.sub(r"\n", "", xdesc)
            s = re.sub(r"\t", "", s)
            new_xdesc = s

            s2 = re.sub(r"\n", "", xtitle)
            s2 = re.sub(r"\t", "", s2)
            new_xtitle = s2


            ulta_cleansers.append(new_xtitle + " " + new_xdesc)
        
        except:  
            #pdb.set_trace()
            continue
        

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [9]:
ulta_cleansers

['Redken All Soft Conditioner',
 'Paul Mitchell Tea Tree Special Shampoo',
 'Redken All Soft Shampoo',
 'Matrix Biolage Colorlast Conditioner',
 'Matrix Biolage Colorlast Shampoo',
 'Paul Mitchell Tea Tree Lavender Mint Moisturizing Conditioner',
 'Paul Mitchell Tea Tree Lavender Mint Moisturizing Shampoo',
 'Matrix Biolage Ultra Hydrasource Conditioner',
 'Redken Color Extend Magnetics Shampoo',
 'Pureology Hydrate Conditioner',
 'Pureology Hydrate Shampoo',
 'Redken Color Extend Blondage Color Depositing Purple Shampoo',
 'Redken Color Extend Shampoo',
 'Redken Color Extend Magnetics Conditioner',
 'Redken Extreme Conditioner',
 'Joico Color Balance Purple Shampoo',
 'Redken Color Extend Conditioner',
 'Paul Mitchell Tea Tree Special Conditioner',
 'Redken Extreme Shampoo',
 'AG Hair Moisture Fast Food Leave-On Conditioner',
 'Redken Color Extend Blondage Color Depositing Purple Conditioner',
 'Matrix Biolage Hydrasource Shampoo',
 "It's A 10 Miracle Deep Conditioner Plus Keratin",
 

In [10]:
len(ulta_cleansers)

1460

In [ ]:
url = f'https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=96&Nrpp=9'
html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
print(html_page.status_code())
soup = BeautifulSoup(html_page.content, 'html.parser')
soup.prettify()
pdb.set_trace()
# xdesc = soup.find_all("p", class_='prod-desc')[8].text
# xtitle = soup.find_all("h4", class_="prod-title")[8].text
# print((xdesc))
# print((xtitle))

In [11]:
Ulta_products = ulta_cleansers + ulta_styling

In [12]:
with open('../../data/02_intermediate/Ulta_products.pkl', 'wb') as f:
    pickle.dump(Ulta_products, f)

In [13]:
# with open('parrot.pkl', 'rb') as f:
# ...   mynewlist = pickle.load(f)

---

## Sallys Scrape
### Exploration of the soup

## Target Soup
#### Just looking into some things

In [37]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://www.target.com/c/natural-hair-care/-/N-4smdr?lnk=NaturalHairCare', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
soup.prettify

<bound method Tag.prettify of <!DOCTYPE doctype html>
 <html lang="en"> <head> <meta charset="utf-8"/> <meta content="IE=edge" http-equiv="x-ua-compatible"/> <meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=2" name="viewport"/> <link crossorigin="" href="https://target.scene7.com" rel="preconnect"/> <link crossorigin="" href="https://assets.targetimg1.com" rel="preconnect"/> <script>!function(){if("PerformanceLongTaskTiming"in window){var n=window.__tti={e:[]};n.o=new PerformanceObserver(function(e){n.e=n.e.concat(e.getEntries())}),n.o.observe({entryTypes:["longtask"]})}}()</script> <script>!function(c,f){var t,o,i,e=[],r={passive:!0,capture:!0},n=new Date,a="pointerup",u="pointercancel";function p(n,e){t||(t=e,o=n,i=new Date,w(f),s())}function s(){0<=o&&o<i-n&&(e.forEach(function(n){n(o,t)}),e=[])}function l(n){if(n.cancelable){var e=(1e12<n.timeStamp?new Date:performance.now())-n.timeStamp;"pointerdown"==n.type?function(n,e){function t(){p(n,e),i()}func

In [19]:
'https://www.target.com/c/natural-hair-care/-/N-4smdr?lnk=NaturalHairCare'
'https://www.target.com/c/natural-hair-care/-/N-4smdr?lnk=NaturalHairCare&Nao=24'
'https://www.target.com/c/natural-hair-care/-/N-4smdr?lnk=NaturalHairCare&Nao=48'

'https://www.target.com/c/natural-hair-care/-/N-4smdr?lnk=NaturalHairCare&Nao=48'

In [57]:
xdesc = soup.find_all(attrs={'data-test" : "product-title"})
xdesc

SyntaxError: EOL while scanning string literal (<ipython-input-57-dc5f2d4e593f>, line 1)

In [61]:
xdesc = soup.find_all(attrs={"data-test" : "product-title"})
xdesc

[]

In [40]:
for link in soup.find_all('a'):
    print(link.get('href'))

#content
#footer
/c/shop-all-categories/-/N-5xsxf
/c/women/-/N-5xtd3
/c/men/-/N-18y1l
/c/young-adult/-/N-qh1tf
/c/kids/-/N-xcoz4
/c/shoes/-/N-55b0t
/c/baby/-/N-5xtly
/c/home/-/N-5xtvd
/c/patio-garden/-/N-5xtq9
/c/furniture/-/N-5xtnr
/c/kitchen-dining/-/N-hz89j
/c/toys/-/N-5xtb0
/c/electronics/-/N-5xtg6
/c/video-games/-/N-5xtg5
/c/movies-music-books/-/N-5xsxe
/c/sports-outdoors/-/N-5xt85
/c/beauty/-/N-55r1x
/c/personal-care/-/N-5xtzq
/c/health/-/N-5xu1n
/c/household-essentials/-/N-5xsz1
/c/pets/-/N-5xt44
/c/grocery/-/N-5xt1a
/c/luggage/-/N-5xtz1
/c/school-office-supplies/-/N-5xsxr
/c/party-supplies/-/N-5xt3c
/c/bullseye-s-playground/-/N-tr36l
/c/clearance/-/N-5q0ga
/c/holiday-shop/-/N-4ydi5
/c/target-finds/-/N-0htu7
/c/gifting/-/N-58d7y

/c/clearance/-/N-5q0ga
https://weeklyad.target.com/
/c/top-deals/-/N-4xw74
/c/redcard-exclusive-extras-early-access/-/N-76udn
/c/download-app/-/N-4th2r
/gift-registry/?lnk=nav_menu_registries

/c/redcard/-/N-4tfyn
/c/target-restock/-/N-amb4n?mode=box
/c

In [ ]:
Love Beauty &amp; Planet Coconut Water &amp; Mimosa Flower Volume &amp; Bounty Conditioner - 13.5 fl oz

In [41]:
tag.name


NameError: name 'tag' is not defined